In [ ]:
########################################################################
########################################################################
#                   SOCKETS
########################################################################
########################################################################

import cv2
import numpy as np
import numpy.lib.recfunctions as nlr
from struct import *

#fd = open('C:\\Brandon\\CurrentProjects\\CMUD\\YoloNerf\\output\\framedump.txt', 'rb')
#fd = open('/home/fri/bkynoch/CMUD/YoloNerf/output/framedump.txt', 'rb')
#FRAME_WIDTH = 1000
#FRAME_HEIGHT = 500
#f = np.fromfile(fd, dtype=np.uint8, count=FRAME_WIDTH*FRAME_HEIGHT*3)
#fd.close()

def reshapeBuffer(data, width, height):
    img = data.reshape((height, width, 3))
    img = img * 1.0/255.0
    img = np.flip(img) # Convert from BGR to RGB
    img = np.flip(img, axis=1)
#     img = np.transpose(img, (1, 0, 2))
    # print(img.shape)
    return img


def send_msg(sock, msg):
    # Prefix each message with a 4-byte length (network byte order)
    msg = pack('I', len(msg)) + msg
    sock.send(msg)

def recvall(sock, n):
    # Helper function to recv n bytes or return None if EOF is hit
    data = bytearray()
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data.extend(packet)
    return data


In [6]:
########################################################################
########################################################################
#                   YOLO
########################################################################
########################################################################

import torch
import numpy as np
import cv2
import pafy
from time import time


class ObjectDetection:
    def __init__(self):
        self.model = self.load_model()
        self.classes = self.model.names
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("\n\nDevice Used:",self.device)

    def load_model(self):
        model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        return model

    def score_frame(self, frame):
        """
        Takes a single frame as input, and scores the frame using yolo5 model.
        :param frame: input frame in numpy/list/tuple format.
        :return: Labels and Coordinates of objects detected by model in the frame.
        """
        self.model.to(self.device)
        frame = [frame]
        results = self.model(frame)
     
        labels, cord = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]
        return labels, cord


    def class_to_label(self, x):
        """
        For a given label value, return corresponding string label.
        :param x: numeric label
        :return: corresponding string label
        """
        return self.classes[int(x)]


    def plot_boxes(self, results, frame):
        """
        Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
        :param results: contains labels and coordinates predicted by model on the given frame.
        :param frame: Frame which has been scored.
        :return: Frame with bounding boxes and labels ploted on it.
        """
        labels, cord = results
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
            row = cord[i]
            if row[4] >= 0.2:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                bgr = (0, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
                cv2.putText(frame, self.class_to_label(labels[i]), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)

        return frame


    def inference(self, img_in):
        start_time = time()
        results = self.score_frame(img_in)
        frame = self.plot_boxes(results, img_in)
        end_time = time()
        fps = 1 / np.round(end_time - start_time, 3)
        print(f"Frames Per Second : {fps}")
        cv2.putText(frame, f'YOLO-FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        #cv2.imshow("img", frame)
        #cv2.waitKey(0)
        return frame



########################################################################
########################################################################
#                   YOLO USAGE
########################################################################
########################################################################

#detection = ObjectDetection()

#img = cv2.imread('C:/Brandon/CurrentProjects/CMUD/instant-ngp/data/nerf/DeskM/images/DESK-10.jpg')
#img_out = detection.inference(img)

#cv2.imshow("Detection", img_out)
#cv2.waitKey(0)

Using cache found in C:\Users\Brandon/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-9-22 Python-3.10.4 torch-1.12.1 CUDA:0 (NVIDIA GeForce GTX 980 Ti, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 




Device Used: cuda
Frames Per Second : 12.987012987012987


-1

In [ ]:
########################################################################
########################################################################
#                   MAIN
########################################################################
########################################################################

import socket
from struct import *

HOST = "127.0.0.1" #Localhost / loopback
PORT = 28508

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((HOST, PORT))

yolo_detection = ObjectDetection()

while (True):
    # Fetch and decode package header
    package_header = s.recv(8)
    frame_dimension = unpack('II', package_header)
    print(frame_dimension)
    width = frame_dimension[0]
    height = frame_dimension[1]

    # Fetch payload
    buffer = recvall(s, width * height * 3)
    print(len(buffer))
    send_msg(s, b'Received buffer') # Send response immediately to unblock thread in C++

    np_buffer = np.frombuffer(buffer, dtype=np.uint8)

    img = reshapeBuffer(np_buffer, width, height)
    img_out = yolo_detection.inference(img)

    cv2.imshow('Framedump', img_out)
    cv2.waitKey(32)


In [ ]:
test_img = np.zeros([500,500,3])

test_img[:,:,2] = np.ones([500,500])*0/255.0 #R
test_img[:,:,1] = np.ones([500,500])*0/255.0 #G
test_img[:,:,0] = np.ones([500,500])*255/255.0 #B

#cv2.imwrite('color_img.jpg', test_img)
cv2.imshow("image", test_img)
cv2.waitKey()